# API Center ❤️ all APIs

## Import from Generic OpenAPI lab
![image](../../images/import-from-generic-openapi.png)

Playground to experiment importing APIs in the OpenAPI specification into API Center. We start by creating an API Center instance using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [Azure CLI]((https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest)) to import the APIs.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to:
- Deploy Azure API Center using Bicep.
- Configure the API Center metadata scheme for APIs, Environments and Deployments.
- Understand the Azure CLI commands to manage Azure API Center. [Full list of commands available here](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest).
- Create an enviroment with custom metadata properties 
- Register an API using the OpenAPI spec as the source of truth and assign API metadata properties.
- Register an API with individual commands to have full control over all the properties. 
- Search and discover the APIs registered in API Center.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)


### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

# the following metadata will be created during the deployment
$metadata = @(
  @{
    "name" = 'termsOfService'
    "schema" = '{"title":"termsOfService","description":"The terms of service URL for using the API","type":"string","format":"uri"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'externalDocs'
    "schema" = '{"title":"externalDocs","description":"External Documents that describe the API","type":"object","properties":{"description":{"title":"description","description":"Description of the External Documents","type":"string"},"url":{"title":"url","description":"The URL to reference the external documents","type":"string","format":"uri"}},"required":["url"]}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'sourceControl'
    "schema" = '{"title":"sourceControl","description":"Source Control Info","type":"object","properties":{"type":{"type":"string","title":"type","description":"Source Control System type","oneOf":[{"const":"GitHub","description":""},{"const":"GitLab","description":""},{"const":"Bitbucket","description":""},{"const":"Other","description":""}]},"repository":{"title":"repository","description":"URL for the repository","type":"string","format":"uri"},"url":{"title":"url","description":"Full URL for the API definition stored in the source control system","type":"string","format":"uri"}},"required":["type","repository","url"]}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'securitySchemes'
    "schema" = '{"title":"securitySchemes","description":"API Security Schemes","type":"array","items":{"type":"object","properties":{"name":{"title":"name","description":"Name of the API Security Scheme","type":"string"},"type":{"type":"string","title":"type","description":"The Type of the API Security Scheme","oneOf":[{"const":"http","description":"Basic, Bearer and other HTTP authentications schemes"},{"const":"apiKey","description":"API keys and cookie authentication"},{"const":"oauth2","description":"OAuth 2.0 is the industry-standard protocol for authorization."},{"const":"openIdConnect ","description":"OpenID Connect Discovery"}]},"authorizationurl":{"title":"authorizationUrl","type":"string","format":"uri"}},"required":["name","type"]}}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'host'
    "schema" = '{"title":"host","description":"The name of the host that is hosting the API","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  }
)

# the following environment(s) will be created during the deployment
$environments = @(
  @{
    "title" = 'prod'
    "kind" = 'production'
    "description" = 'Production Environment'
    "server" = @{
        "type" = 'Kubernetes' 
        "managementPortalUri" = 'https://swagger.io/tools/swaggerhub/'
    }
    "onboarding" = @{
        "developerPortalUri" = 'https://swagger.io/tools/swaggerhub/features/swaggerhub-portal/'
        "instructions" = 'Sign in and access the developer portal to get started with the API'
    }
    "customProperties" = @{ 
    }
  }
)

$environmentCustomProperties = @{
    "host" = "petstore3.swagger.io"
}

$apiCustomProperties = @{
    "termsOfService" = "http://swagger.io/terms/"
    "externalDocs" = @{
        "description" = "Find out more about Swagger"
        "url" = "http://swagger.io"
    }
    "sourceControl" = @{
        "type" = "GitHub"
        "repository" = "https://github.com/swagger-api/swagger-petstore"
        "url" = "https://github.com/swagger-api/swagger-petstore/blob/master/src/main/resources/openapi.yaml"
    }
}

$deploymentCustomProperties = @{
    "securitySchemes" = @(
        @{
          "name" = "petstore_auth"
          "type" = "oauth2"
          "authorizationurl" = "https://petstore3.swagger.io/oauth/authorize"
        }
    )
}

Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Update the environment metadata

We will [update the enviroment](https://learn.microsoft.com/en-us/cli/azure/apic/environment?view=azure-cli-latest#az-apic-environment-update) with the `custom properties` defined in the variable `environmentCustomProperties`

In [ ]:
$environmentCustomPropertiesJson = ConvertTo-Json -InputObject $environmentCustomProperties -Depth 10 -Compress
az apic environment update -g $resourceGroupName -s $apicResourceName --environment-id "prod" `
            --custom-properties $environmentCustomPropertiesJson.replace('"','\"')

### 5️⃣. 1 Register an API with the register command

[Registers](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-register) a new API with version, definition, and associated deployments using the specification file as the source of truth.

In [ ]:
az apic api register -g $resourceGroupName -s $apicResourceName --api-location "petstore.yaml" --environment-name prod

### 5️⃣. 2 Update the API metadata

[Updates the API](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-update) with custom properties defined in the `apiCustomPropertiesJson` variable. 

In [ ]:
$apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
az apic api update -g $resourceGroupName -s $apicResourceName --api-id petstore --custom-properties $apiCustomPropertiesJson.replace('"','\"')

### 6️⃣. 1 Register another API with custom metadata using individual commands

With the [`create` command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-create) we will add a new API identified by the `api-id` parameter, and details such as `title`, `contacts`, `license`, `summary` and `custom properties`.

The `type` accepted values are: graphql, grpc, rest, soap, webhook, websocket

In [ ]:
 $apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
 az apic api create -g $resourceGroupName -s $apicResourceName --api-id "another-petstore" --type REST `
       --title "Another petstore" --description "This is a sample Pet Store Server based on the OpenAPI 3.0 specification." `
       --contacts '[{\"name\":\"apiteam\",\"email\":\"apiteam@swagger.io\"}]' `
       --license '{\"name\":\"Apache 2.0\",\"url\":\"http://www.apache.org/licenses/LICENSE-2.0.html\"}' `
       --summary "This is a sample Pet Store Server based on the OpenAPI 3.0 specification." `
       --custom-properties $apiCustomPropertiesJson.replace('"','\"')

### 6️⃣. 2 Add a version to the API

An API can have multiple versions. We will [create a new version](https://learn.microsoft.com/en-us/cli/azure/apic/api/version?view=azure-cli-latest#az-apic-api-version-create) identified by the `version-id`, with a `title` and a `lifecycle-stage`.

The `lifecycle-stage` can have one of the following values:
- design
- development
- testing
- preview
- production
- deprecated
- retired

In [ ]:
az apic api version create -g $resourceGroupName -s $apicResourceName --api-id "another-petstore" --version-id "1-0-11" --title "1.0.11" `
      --lifecycle-stage "production"

### 6️⃣. 3 Add a definition to the API version

An API version can have multiple definitions. We will [create a new definition](https://learn.microsoft.com/en-us/cli/azure/apic/api/definition?view=azure-cli-latest#az-apic-api-definition-create) identified by the `definition-id`, with a `title` and a `description`.

In [ ]:
az apic api definition create -g $resourceGroupName -s $apicResourceName --api-id "another-petstore" `
    --version-id "1-0-11" --definition-id "openapi" --title "OpenAPI" --description "OpenAPI spec"

### 6️⃣. 4 Import the specification to the API version definition

Now we will [import the specification](https://learn.microsoft.com/en-us/cli/azure/apic/api/definition?view=azure-cli-latest#az-apic-api-definition-import-specification) by setting the `specification` property with `name` and `version`.

The allowed values for the `specification name` are: 
- OpenAI
- WSDL
- WADL
- GraphQL
- gRPC
- AsyncAPI
- RAML
- Other

In [ ]:
az apic api definition import-specification -g $resourceGroupName -s $apicResourceName `
        --api-id "another-petstore" --version-id "1-0-11" --definition-id "openapi" `
        --specification '{\"name\":\"openapi\",\"version\":\"3.0.3\"}' `
        --format "link" --value 'https://raw.githubusercontent.com/swagger-api/swagger-petstore/master/src/main/resources/openapi.yaml'

### 6️⃣. 5 Create a deployment to link the API version definition with an environment

To [create a deployment](https://learn.microsoft.com/en-us/cli/azure/apic/api/deployment?view=azure-cli-latest#az-apic-api-deployment-create) we specify the `deployment-id`, `title`, `description`, `api-id`, `environment-id`, `server properties` and `custom properties`

In [ ]:
$deploymentCustomPropertiesJson = ConvertTo-Json -InputObject $deploymentCustomProperties -Depth 10 -Compress
az apic api deployment create -g $resourceGroupName -s $apicResourceName --deployment-id "prod" `
        --title "Production deployment" --description "Petstore production deployment." --api-id "another-petstore" `
        --environment-id "/workspaces/default/environments/prod" `
        --definition-id "/workspaces/default/apis/another-petstore/versions/1-0-11/definitions/openapi" `
        --server '{\"runtimeUri\":[\"https://petstore3.swagger.io/api/v3\"]}' `
        --custom-properties $deploymentCustomPropertiesJson.replace('"','\"')

### 7️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [27]:
az apic api list -g $resourceGroupName -s $apicResourceName --query "[?title == 'petstore'].{Name:name, Title:title, Kind:kind, ContactEmail:contacts[0].email}" -o table

Name      Title     Kind    ContantEmail
--------  --------  ------  ------------------
petstore  petstore  rest    apiteam@swagger.io


### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.